In [4]:
%reload_ext autoreload
%autoreload 2
#%matplotlib notebook

In [5]:
import alphatims.bruker
import alphatims.utils
import alphatims.plotting
import alphatims.tempmmap as tm

import alphadia.calibration
import alphadia.preprocessing
import alphadia.annotation
import alphadia.library

import numpy as np
import pandas as pd
import scipy.ndimage
import colorcet
import hvplot.pandas
from matplotlib import pyplot as plt
import alphabase.io.hdf
import tqdm
import sklearn.linear_model
import functools
import holoviews as hv
import seaborn as sns
import h5py
import sklearn.neighbors

import peptdeep.rescore.percolator
from peptdeep.settings import global_settings as peptdeep_global_settings
import os

alphatims.utils.set_threads(7)
alphatims.utils.set_logger()
peptdeep_global_settings['percolator']['multiprocessing'] = False

In [6]:
%%time

library = alphadia.annotation.library.Library()
library.import_from_file(
    "/Users/georgwallmann/Documents/data/libraries/2022_09_06_human_trypsin_decoy.hdf"
)

dia_data_sets = ['/Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.d']
dia_data = alphatims.bruker.TimsTOF(dia_data_sets[0])



2022-10-14 16:11:33> Loading library
2022-10-14 16:11:36> Importing data from /Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.d
2022-10-14 16:11:36> Using .d import for /Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.d
2022-10-14 16:11:36> Reading frame metadata for /Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.d
2022-10-14 16:11:36> Reading 19,827 frames with 764,964,665 detector events for /Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.d


100%|██████████| 19827/19827 [00:17<00:00, 1144.85it/s]

2022-10-14 16:11:54> Indexing /Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.d...
2022-10-14 16:11:54> Bruker DLL not available, estimating mobility values
2022-10-14 16:11:54> WARNING: Acquisition software is Bruker otofControl, mz min/max values are assumed to be 5 m/z wider than defined in analysis.tdf!
2022-10-14 16:11:54> Bruker DLL not available, estimating mz values
2022-10-14 16:11:54> Indexing quadrupole dimension


2022-10-14 16:11:54> Successfully imported data from /Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.d
CPU times: user 14.3 s, sys: 5.33 s, total: 19.6 s
Wall time: 20.7 s


In [ ]:
preprocessing_workflow = alphadia.preprocessing.Workflow()
preprocessing_workflow.set_dia_data(dia_data)
preprocessing_workflow.run_default()
preprocessing_workflow.save_to_hdf(file_name = '/Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.extracted.msms.hdf')

In [7]:
preprocessing_workflow = alphadia.preprocessing.Workflow()
preprocessing_workflow.set_dia_data(dia_data)
preprocessing_workflow.load_from_hdf(file_name = '/Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.extracted.msms.hdf')

2022-10-14 16:11:59> Loading preprocessing workflow from /Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944.extracted.msms.hdf.


In [8]:
annotator = alphadia.annotation.Annotator()
annotator.set_preprocessor(preprocessing_workflow)
annotator.set_library(library)
annotator.run_default()


2022-10-14 16:12:16> Quick library annotation of mono isotopes with self.ppm_mean=0 and self.precursor_ppm=50
2022-10-14 16:12:16> PSMs to test: 95432193


100%|██████████| 139381/139381 [00:40<00:00, 3474.69it/s]


2022-10-14 16:13:00> Estimating ppm values
2022-10-14 16:13:00> Quick library annotation of mono isotopes with self.ppm_mean=34.235617330848406 and self.precursor_ppm=28.151834948407778
2022-10-14 16:13:00> PSMs to test: 66199342


100%|██████████| 139381/139381 [00:22<00:00, 6334.44it/s] 


2022-10-14 16:13:22> Appending stats to quick annotation


...

2022-10-14 16:13:25> Percolating PSMs
2022-10-14 16:13:25> Calculating quick log odds


2022-10-14 16:13:27> Found 4424 targets for calibration
2022-10-14 16:13:27> Using selected "log_odds" as initial_feature
2022-10-14 16:13:27> Training & cross-validation on 1215 targets and 3329 decoys


/Users/georgwallmann/Documents/git/alphadia/alphadia/annotation/percolation.py:344: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfT_high = df[
/Users/georgwallmann/Documents/git/alphadia/alphadia/annotation/percolation.py:363: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_training = dfT_high.append(
/Users/georgwallmann/miniconda3/envs/alphadia/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/georgwallmann/miniconda3/envs/alphadia/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/User

2022-10-14 16:13:34> The best parameters selected by 5-fold cross-validation were {'clf__max_depth': 25, 'clf__max_leaf_nodes': 150}
2022-10-14 16:13:34> The train accuracy was 0.8413292253521126
2022-10-14 16:13:34> Testing on 4860 targets and 13320 decoys
2022-10-14 16:13:34> The test accuracy was 0.7649064906490649
2022-10-14 16:13:34> Top features {'rt_diff_calibrated': 0.17825056151157576, 'relative_found_int': 0.08553685167875233, 'ppm_diff_calibrated': 0.06452976433749583, 'relative_found_y_int': 0.06408345125966412, 'candidates': 0.059432876152320124, 'im_diff_calibrated': 0.05893040269484731, 'relative_missed_int': 0.05647529626493989, 'frequency_counts': 0.056302478491772605, 'total_peaks': 0.04502413172906012, 'nAA': 0.043118253017638136, 'y_mean_ppm': 0.041759509762931114, 'b_mean_ppm': 0.0382719549211844, 'relative_missed_y_int': 0.037793922148036806, 'relative_found_b_int': 0.03675185507882074, 'relative_missed_b_int': 0.03265351575073422, 'y_hit_counts': 0.02606920358702

/Users/georgwallmann/miniconda3/envs/alphadia/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/georgwallmann/Documents/git/alphadia/alphadia/annotation/percolation.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.score_df["target_type"][


In [16]:
temp_folder = '/Users/georgwallmann/Documents/data/brunner2022/diaPASEF_repetitions/20200827_TIMS04_EVO07_AnBr_1ng_dia_rep01_400s_30min_S1-D1_1_2944_temp'
annotator.percolator.save_to_tsv(os.path.join(temp_folder, 'spectrum_hits.tsv'), run_name=dia_data.sample_name)
annotator.percolator.save_to_hdf(os.path.join(temp_folder, 'spectrum_hits.hdf'), run_name=dia_data.sample_name)